需求说明：
    找出polymarket上 strike date的3天前和7天前市场发生概率大于0.9，小于0.95，且在strike date 当日发生成交的概率

分解需求：
    1、从文档(polymarket\markets_data.csv) 找出 已经关闭的事件，字段closed
    2、获取已关闭事件的所有市场的token_id。并获取历史价格数据，全部存入对应question_id
    3、找到在 strike date 之前的倒数第三天和倒数第7天的市场发生概率大于0.9且小于0.95的事件及其市场，计算总的事件数N
    4、找到第3、的市场，看其是否在strike date 当日的概率为1， 或者是否为winner。计算win的总事件的数目m
    5、计算总发生概率 m/N，并输出总结果

In [197]:
import pandas as pd
import numpy as np
import csv
import json
import os
from py_clob_client.client import ClobClient,BookParams
from py_clob_client.constants import POLYGON
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

import requests
from datetime import datetime
import re

1、从文档('polymarket\markets_data.csv') 找出已经关闭的事件，字段closed

In [5]:
poly_data = pd.read_csv('polymarket\markets_data.csv')

In [7]:
poly_data.columns

Index(['accepting_order_timestamp', 'accepting_orders', 'active', 'archived',
       'closed', 'condition_id', 'description', 'enable_order_book',
       'end_date_iso', 'fpmm', 'game_start_time', 'icon', 'image',
       'is_50_50_outcome', 'maker_base_fee', 'market_slug',
       'minimum_order_size', 'minimum_tick_size', 'neg_risk',
       'neg_risk_market_id', 'neg_risk_request_id', 'notifications_enabled',
       'question', 'question_id', 'rewards', 'seconds_delay', 'tags',
       'taker_base_fee', 'token_outcome', 'token_price', 'token_token_id',
       'token_winner', 'tokens'],
      dtype='object')

In [24]:
# poly_data['token_winner']

In [268]:
# poly_data_closed=poly_data[poly_data['closed']==True][['condition_id','question','question_id','tokens','token_winner']].reset_index(drop=True)

In [269]:
poly_data_closed = poly_data

2、获取已关闭事件的所有市场的token_id，并获取历史价格数据，全部存入对应question_id

In [132]:
clob_endpoint = "https://clob.polymarket.com"
url = f"{clob_endpoint}/prices-history" # 获取历史价格

In [260]:
i

13519

In [258]:
js_str

'[{"token_id": "75674010897912433263470070979110319954823328111562242389419248656674366188074", "outcome": "Gillen ", "price": 1, "winner": true}, {"token_id": "23682917493470547014927814926256831892437243307768785697446430339870292046840", "outcome": "D"Esposito", "price": 0, "winner": false}]'

In [ ]:
pattern = r'(?<=[a-zA-Z0-9])"s'

re.sub(pattern,'',)

In [264]:
len(tokens_id)

2

In [ ]:
pro_path = '2024-11-15/price_history/'
questions_id = []
for i in range(poly_data_closed.shape[0]):
    tmp_data = poly_data_closed.iloc[i,:]
    que_id = tmp_data['question_id']
    os.makedirs(pro_path+str(que_id), exist_ok=True)

    questions_id.append(que_id)

    # 获取token_id
    js_str = tmp_data['tokens'].replace("'",'"')
    if 'False' in js_str:  # 如果str中包含False 或者 True, 必须转化为小写
        js_str=js_str.replace('False','false')

    if 'True' in js_str:
        js_str=js_str.replace('True','true')

    # json字符串的处理1
    pattern = r'(?<=[a-zA-Z0-9])"s'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
    js_str = re.sub(pattern, '', js_str)    

    # json字符串的处理2
    pattern = r'(?<=[a-zA-Z0-9])"N'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
    js_str = re.sub(pattern, '', js_str) 

    # json字符串的处理3
    pattern = r'(?<=[a-zA-Z0-9])"E'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
    js_str = re.sub(pattern, '', js_str)  

    tokens_id=json.loads(js_str)

    if len(os.listdir(pro_path+str(que_id)))==len(tokens_id):
        continue

    # 获取历史价格数据
    for j in range(len(tokens_id)):
        token_id = tokens_id[j]['token_id']
        params = {
            "market": token_id,  # 替换为实际的CLOB token id
            "interval": "max",  # 获取最大长度的数据
            "fidelity": 60*24  # 数据分辨率，单位为分钟
        }
        try:
            # 发送GET请求
            response = requests.get(url, params=params)
            response.raise_for_status()  # 如果响应状态码不是200，将抛出异常
            # 解析响应数据
            prices_history = response.json()
            prices_history = pd.DataFrame(prices_history['history'])
            if not prices_history.empty:
                prices_history['t'] = prices_history['t'].apply(lambda x:datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
                file_path = f'{pro_path}/{que_id}/{token_id}.csv'
                # print(file_path)
                prices_history.to_csv(file_path, index=False)
        except requests.RequestException as e:
            print(f"请求错误：{e}")
            print(i,que_id)
        
        

请求错误：400 Client Error: Bad Request for url: https://clob.polymarket.com/prices-history?market=&interval=max&fidelity=1440
1 nan
请求错误：400 Client Error: Bad Request for url: https://clob.polymarket.com/prices-history?market=&interval=max&fidelity=1440
1 nan
请求错误：400 Client Error: Bad Request for url: https://clob.polymarket.com/prices-history?market=&interval=max&fidelity=1440
29 nan
请求错误：400 Client Error: Bad Request for url: https://clob.polymarket.com/prices-history?market=&interval=max&fidelity=1440
29 nan
请求错误：400 Client Error: Bad Request for url: https://clob.polymarket.com/prices-history?market=&interval=max&fidelity=1440
54 nan
请求错误：400 Client Error: Bad Request for url: https://clob.polymarket.com/prices-history?market=&interval=max&fidelity=1440
54 nan
请求错误：400 Client Error: Bad Request for url: https://clob.polymarket.com/prices-history?market=&interval=max&fidelity=1440
61 nan
请求错误：400 Client Error: Bad Request for url: https://clob.polymarket.com/prices-history?market=&inte

In [ ]:
# 计算总样本数
pro_path = '2024-11-15/price_history/'
X=0
for i in os.listdir(pro_path):
    if len(os.listdir(pro_path+'/'+i))>=2:
        X+=1

In [ ]:
X

3、找到在 strike date 前 的倒数第三天和倒数第7天的市场发生概率大于0.9且小于0.95的事件及其市场，计算总的事件数N

In [ ]:
# pro_path = '2024-11-15/limited_events/'
events_id = [] # 存入满足条件的 question_id
markets_id = [] # 存入满足条件的 token_id
N = 0 # 记录满足条件的总事件数目
flag = False  # flag = False 表示该事件的市场中没有满足条件的市场
for file_path in os.listdir('2024-11-15/price_history/'):
    que_id = file_path

    for f_p in os.listdir(f'2024-11-15/price_history/{que_id}'):
        prices_history = pd.read_csv(f'2024-11-15/price_history/{que_id}/{f_p}')

        # strike date 前 倒数第三天的价格
        pre_3d = prices_history.iloc[-4,:]
        price_pre_3d = pre_3d['p']

        # strike date 前 倒数第7天的价格
        pre_7d = prices_history.iloc[-8,:]
        price_pre_7d = pre_7d['p']

        if 0.9<pre_3d<0.95 and 0.9<pre_7d<0.95:
            markets_id.append(f_p[:-4])
            flag=True
    
    if flag == True:
        N+=1
        events_id.append(que_id)
    
    flag=False



In [ ]:
events_id+'/'+markets_id

4、找到第3、的市场，看其是否在strike date 当日成交。计算总成交事件的数目m

In [ ]:
# 为确保准确，应该找到交易结束的时间
for que_id in events_id:
    tmp_data = poly_data_closed[poly_data_closed['question_id']==que_id]
    
    js_str = tmp_data['tokens'].replace("'",'"')
    if 'False' in js_str:  # 如果str中包含False 或者 True, 必须转化为小写
        js_str=js_str.replace('False','false')

    if 'True' in js_str:
        js_str=js_str.replace('True','true')

    # json字符串的处理1
    pattern = r'(?<=[a-zA-Z0-9])"s'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
    js_str = re.sub(pattern, '', js_str)    

    # json字符串的处理2
    pattern = r'(?<=[a-zA-Z0-9])"N'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
    js_str = re.sub(pattern, '', js_str) 

    # json字符串的处理3
    pattern = r'(?<=[a-zA-Z0-9])"E'  # 匹配s前面的", 且该"前面的字符只能是数字或者字母
    js_str = re.sub(pattern, '', js_str)  

    tokens_id=json.loads(js_str)

    if len(os.listdir(pro_path+str(que_id)))==len(tokens_id):
        continue

    # 获取win是否为True
    for j in range(len(tokens_id)):
        token=tokens_id[j]['win']